<a href="https://colab.research.google.com/github/vz415/AE32D_optimization/blob/master/AE32D_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import itertools

import os
from scipy import io

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from tqdm import tqdm
%matplotlib inline

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Loading up data from cubes
data_dir = "./drive/My Drive/Lab/BMP AE Models/Data"
data_file1 = 'EQTK_322_S_20171115_030058.mat'
data_file2 = 'EQTK_322_S_20200608_184021.mat'
model_size = tuple([int(i) for i in data_file1[5:8]])
data1 = io.loadmat(os.path.join(data_dir, data_file1))
data2 = io.loadmat(os.path.join(data_dir, data_file2))
S1 = data1['S']
S2 = data2['S']
S = np.concatenate((S1, S2.reshape((1000,1728))), axis=0)

In [ ]:
# !pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime, os

In [ ]:
# Getting classy
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
import h5py
import os
import pathlib
# from keras.callbacks import TensorBoard

# Scaling data as was used for PCA
# scaler = MinMaxScaler()
# scaler = StandardScaler()
# x_test = scaler.fit_transform(S[:150,:])
# x_train = scaler.fit_transform(S[150:,:])
x_test = S[:200,:]
x_train = S[200:,:]

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 54, assuming input is 1728 floats

# this is our input placeholder
input_img = Input(shape=(1728,), name='Input')
# "encoded" is the encoded representation of the input
encoded = Dropout(0.05)(input_img)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(256, activation='relu', 
                activity_regularizer=regularizers.l1(10e-4))(encoded)
encoded = Dense(128, activation='relu')(encoded)
# encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input - linear activation for regression
# decoded = Dense(1728, activation='linear')(encoded)
# decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dropout(0.05)(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(1728, activation='linear')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# Configuring optimizer and loss function
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, momentum=0)
autoencoder.compile(optimizer=opt, 
                    loss=tf.keras.losses.MeanSquaredError())

# Early stopping and checkpointing
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6000)
pathlib.Path("./drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 4/30062020/AE_32D_rmsprop").mkdir(parents=True, exist_ok=True)
drive_file = './drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 4/30062020/AE_32D_rmsprop/AE_drop0-05_512_256_L1-0-1e-4_128_32_128_drop0-05_256_512_model.h5'
mc = ModelCheckpoint(filepath=drive_file, 
                     monitor='val_loss', mode='min', 
                     verbose=0, save_best_only=False) # Don't only save best
csv_logger = CSVLogger(drive_file[:-3]+'history_log.csv', append=True)

# Normalize, using standarad scaler like in PCA, and shuffle
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Logging epochs for Tensorboard visualization
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# %tensorboard --port=5036 --logdir $logdir
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Train for 50 epochs
history = autoencoder.fit(x_train, x_train,
                          epochs=40000,
                          batch_size=512,
                          shuffle=True,
                          validation_data=(x_test, x_test),
                          callbacks=[es, mc, csv_logger])

Using TensorFlow backend.


Train on 1800 samples, validate on 200 samples
Epoch 1/40000
1800/1800 [==============================] - 2s 1ms/step - loss: 3381.9402 - val_loss: 1040.7213
Epoch 2/40000
1800/1800 [==============================] - 0s 45us/step - loss: 2089.9846 - val_loss: 604.7260
Epoch 3/40000
 512/1800 [=======>......................] - ETA: 0s - loss: 974.5686

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Streaming output truncated to the last 5000 lines.
Epoch 5612/40000
1800/1800 [==============================] - 0s 44us/step - loss: 66.8958 - val_loss: 68.3476
Epoch 5613/40000
1800/1800 [==============================] - 0s 45us/step - loss: 57.2454 - val_loss: 48.7089
Epoch 5614/40000
1800/1800 [==============================] - 0s 45us/step - loss: 22.9503 - val_loss: 37.7051
Epoch 5615/40000
1800/1800 [==============================] - 0s 43us/step - loss: 18.8013 - val_loss: 45.1063
Epoch 5616/40000
1800/1800 [==============================] - 0s 42us/step - loss: 39.9033 - val_loss: 64.4648
Epoch 5617/40000
1800/1800 [==============================] - 0s 42us/step - loss: 76.5392 - val_loss: 58.1363
Epoch 5618/40000
1800/1800 [==============================] - 0s 45us/step - loss: 27.9114 - val_loss: 56.3936
Epoch 5619/40000
1800/1800 [==============================] - 0s 41us/step - loss: 40.1951 - val_loss: 82.5095
Epoch 5620/40000
1800/1800 [==============================] -

In [ ]:
# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 54, assuming input is 1728 floats

# this is our input placeholder
input_img = Input(shape=(1728,), name='Input')
# "encoded" is the encoded representation of the input
encoded = Dropout(0.05)(input_img)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu', 
                activity_regularizer=regularizers.l1(10e-4))(encoded)
# encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input - linear activation for regression
# decoded = Dense(1728, activation='linear')(encoded)
# decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dropout(0.05)(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(1728, activation='linear')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# Configuring optimizer and loss function
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
autoencoder.compile(optimizer=opt, 
                    loss=tf.keras.losses.MeanSquaredError())

# Early stopping and checkpointing
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6000)
pathlib.Path("./drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 4/30062020/AE_32D_rmsprop").mkdir(parents=True, exist_ok=True)
drive_file = './drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 4/30062020/AE_32D_rmsprop/AE_drop0-05_512_256_128_L1-0-1e-4_32_128_drop0-05_256_512_model.h5'
mc = ModelCheckpoint(filepath=drive_file, 
                     monitor='val_loss', mode='min', 
                     verbose=0, save_best_only=False) # Don't only save best
csv_logger = CSVLogger(drive_file[:-3]+'history_log.csv', append=True)

# Normalize, using standarad scaler like in PCA, and shuffle
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Logging epochs for Tensorboard visualization
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# %tensorboard --port=5036 --logdir $logdir
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Train for 50 epochs
history = autoencoder.fit(x_train, x_train,
                          epochs=40000,
                          batch_size=512,
                          shuffle=True,
                          validation_data=(x_test, x_test),
                          callbacks=[es, mc, csv_logger])

Train on 1800 samples, validate on 200 samples
Epoch 1/40000
1800/1800 [==============================] - 0s 149us/step - loss: 12471.3391 - val_loss: 1677.6456
Epoch 2/40000
1800/1800 [==============================] - 0s 42us/step - loss: 1561.7604 - val_loss: 1537.2352
Epoch 3/40000
 512/1800 [=======>......................] - ETA: 0s - loss: 2153.6311

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Streaming output truncated to the last 5000 lines.
Epoch 7370/40000
1800/1800 [==============================] - 0s 45us/step - loss: 49.7706 - val_loss: 42.0328
Epoch 7371/40000
1800/1800 [==============================] - 0s 41us/step - loss: 10.4813 - val_loss: 45.3576
Epoch 7372/40000
1800/1800 [==============================] - 0s 43us/step - loss: 6.6804 - val_loss: 40.7813
Epoch 7373/40000
1800/1800 [==============================] - 0s 43us/step - loss: 7.1907 - val_loss: 59.9914
Epoch 7374/40000
1800/1800 [==============================] - 0s 41us/step - loss: 40.4099 - val_loss: 83.5547
Epoch 7375/40000
1800/1800 [==============================] - 0s 39us/step - loss: 29.9809 - val_loss: 62.1063
Epoch 7376/40000
1800/1800 [==============================] - 0s 43us/step - loss: 12.0825 - val_loss: 51.8442
Epoch 7377/40000
1800/1800 [==============================] - 0s 41us/step - loss: 13.8477 - val_loss: 93.3967
Epoch 7378/40000
1800/1800 [==============================] - 0

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
import h5py
import os
import pathlib

x_test = S[:200,:]
x_train = S[200:,:]

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 54, assuming input is 1728 floats

# this is our input placeholder
input_img = Input(shape=(1728,), name='Input')
# "encoded" is the encoded representation of the input
encoded = Dropout(0.05)(input_img)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
# encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu', 
                activity_regularizer=regularizers.l1(10e-4))(encoded)

# "decoded" is the lossy reconstruction of the input - linear activation for regression
# decoded = Dense(1728, activation='linear')(encoded)
# decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dropout(0.05)(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(1728, activation='linear')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# Configuring optimizer and loss function
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
autoencoder.compile(optimizer='RMSprop', 
                    loss=tf.keras.losses.MeanSquaredError())

# Early stopping and checkpointing
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6000)
pathlib.Path("./drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 4/30062020/AE_32D_rmsprop").mkdir(parents=True, exist_ok=True)
drive_file = './drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 4/30062020/AE_32D_rmsprop/AE_drop0-05_512_256_128_32_L1-0-1e-4_128_drop0-05_256_512_model.h5'
mc = ModelCheckpoint(filepath=drive_file, 
                     monitor='val_loss', mode='min', 
                     verbose=0, save_best_only=False) # Don't only save best
csv_logger = CSVLogger(drive_file[:-3]+'history_log.csv', append=True)

# Normalize, using standarad scaler like in PCA, and shuffle
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Logging epochs for Tensorboard visualization
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# %tensorboard --port=5036 --logdir $logdir
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Train for 50 epochs
history = autoencoder.fit(x_train, x_train,
                          epochs=40000,
                          batch_size=512,
                          shuffle=True,
                          validation_data=(x_test, x_test),
                          callbacks=[es, mc, csv_logger])

Streaming output truncated to the last 5000 lines.
Epoch 6634/40000
1800/1800 [==============================] - 0s 51us/step - loss: 37.2662 - val_loss: 29.3759
Epoch 6635/40000
1800/1800 [==============================] - 0s 48us/step - loss: 6.5062 - val_loss: 29.0702
Epoch 6636/40000
1800/1800 [==============================] - 0s 41us/step - loss: 5.2753 - val_loss: 27.3903
Epoch 6637/40000
1800/1800 [==============================] - 0s 47us/step - loss: 7.4905 - val_loss: 50.5083
Epoch 6638/40000
1800/1800 [==============================] - 0s 42us/step - loss: 36.3005 - val_loss: 37.1219
Epoch 6639/40000
1800/1800 [==============================] - 0s 42us/step - loss: 10.7543 - val_loss: 43.9377
Epoch 6640/40000
1800/1800 [==============================] - 0s 43us/step - loss: 6.8881 - val_loss: 39.0277
Epoch 6641/40000
1800/1800 [==============================] - 0s 42us/step - loss: 11.0535 - val_loss: 56.9355
Epoch 6642/40000
1800/1800 [==============================] - 0s 

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
import h5py
import os
import pathlib

x_test = S[:200,:]
x_train = S[200:,:]

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 54, assuming input is 1728 floats

# this is our input placeholder
input_img = Input(shape=(1728,), name='Input')
# "encoded" is the encoded representation of the input
encoded = Dropout(0.05)(input_img)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
# encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input - linear activation for regression
# decoded = Dense(1728, activation='linear')(encoded)
# decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(128, activation='relu', 
                activity_regularizer=regularizers.l1(10e-4))(encoded)
decoded = Dropout(0.05)(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(1728, activation='linear')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# Configuring optimizer and loss function
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
autoencoder.compile(optimizer='RMSprop', 
                    loss=tf.keras.losses.MeanSquaredError())

# Early stopping and checkpointing
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6000)
pathlib.Path("./drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 5/30062020/AE_32D_rmsprop").mkdir(parents=True, exist_ok=True)
drive_file = './drive/My Drive/Lab/BMP AE Models/Models/512-256-128-64-32 AE/Dropout Studies 5/30062020/AE_32D_rmsprop/AE_drop0-05_512_256_128_32_128_L1-0-1e-4_drop0-05_256_512_model.h5'
mc = ModelCheckpoint(filepath=drive_file, 
                     monitor='val_loss', mode='min', 
                     verbose=0, save_best_only=False) # Don't only save best
csv_logger = CSVLogger(drive_file[:-3]+'history_log.csv', append=True)

# Normalize, using standarad scaler like in PCA, and shuffle
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Logging epochs for Tensorboard visualization
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# %tensorboard --port=5036 --logdir $logdir
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Train for 50 epochs
history = autoencoder.fit(x_train, x_train,
                          epochs=40000,
                          batch_size=512,
                          shuffle=True,
                          validation_data=(x_test, x_test),
                          callbacks=[es, mc, csv_logger])

Using TensorFlow backend.


Streaming output truncated to the last 5000 lines.
Epoch 8089/40000
1800/1800 [==============================] - 0s 40us/step - loss: 8.1929 - val_loss: 25.6913
Epoch 8090/40000
1800/1800 [==============================] - 0s 45us/step - loss: 10.9501 - val_loss: 26.0660
Epoch 8091/40000
1800/1800 [==============================] - 0s 44us/step - loss: 27.8687 - val_loss: 34.7281
Epoch 8092/40000
1800/1800 [==============================] - 0s 49us/step - loss: 9.0660 - val_loss: 28.8049
Epoch 8093/40000
1800/1800 [==============================] - 0s 41us/step - loss: 8.6186 - val_loss: 24.9077
Epoch 8094/40000
1800/1800 [==============================] - 0s 42us/step - loss: 12.7633 - val_loss: 43.5225
Epoch 8095/40000
1800/1800 [==============================] - 0s 56us/step - loss: 42.4138 - val_loss: 35.0063
Epoch 8096/40000
1800/1800 [==============================] - 0s 46us/step - loss: 11.0437 - val_loss: 24.5098
Epoch 8097/40000
1800/1800 [==============================] - 0s